## used this notebook to test which out of similar summaries or "soup" is produces better suggestions

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
movies = pd.read_csv("re-movies.csv")

In [3]:
movies.head()

,Unnamed: 0,index,id,title,genres,original_language,overview,popularity,production_companies,vote_average,vote_count,credits,keywords,poster_path,recommendations,cast,soup
0,0,0,631842,Knock at the Cabin,"['Horror', 'Mystery', 'Thriller']",en,While vacationing at a remote cabin a young gi...,3422.537,Blinding Edge Pictures-Universal Pictures-Film...,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,"['basedonnovelorbook', 'sacrifice', 'cabin', '...",/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,1058949-646389-772515-505642-143970-667216-104...,"['davebautista', 'jonathangroff', 'benaldridge...",basedonnovelorbook sacrifice cabin faith endof...
1,1,1,646389,Plane,"['Action', 'Adventure', 'Thriller']",en,After a heroic job of successfully landing his...,2618.646,MadRiver Pictures-Di Bonaventura Pictures-G-BA...,6.901,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,"['pilot', 'airplane', 'philippines', 'heldhost...",/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg,505642-758769-864692-631842-1058949-925943-758...,"['gerardbutler', 'mikecolter', 'yosonan', 'ton...",pilot airplane philippines heldhostage planecr...
2,2,2,505642,Black Panther: Wakanda Forever,"['Action', 'Adventure', 'Science Fiction']",en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,2525.408,Marvel Studios,7.338,3922.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,"['lossoflovedone', 'hero', 'sequel', 'superher...",/sv1xJUazXeYqALzczSZ3O6nkH75.jpg,436270-829280-76600-56969-312634-1037858-238-5...,"['letitiawright', ""lupitanyong'o"", 'danaigurir...",lossoflovedone hero sequel superhero basedonco...
3,3,4,315162,Puss in Boots: The Last Wish,"['Animation', 'Adventure', 'Comedy', 'Family']",en,Puss in Boots discovers that his passion for a...,2078.280,DreamWorks Animation-Universal Pictures,8.363,4671.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,"['fairytale', 'talkingdog', 'spinoff', 'afterc...",/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,1033456-1011679-505642-1058949-100088-536554-1...,"['antoniobanderas', 'salmahayek', 'harveyguill...",fairytale talkingdog spinoff aftercreditssting...
4,4,7,937278,A Man Called Otto,"['Comedy', 'Drama']",en,When a lively young family moves in next door ...,1545.382,Playtone-SF Studios-2DUX²-Stage 6 Films-Artist...,7.811,540.0,Tom Hanks-Mariana Treviño-Rachel Keller-Manuel...,"['basedonnovelorbook', 'suicideattempt', 'rema...",/130H1gap9lFfiTF9iDrqNIkFvC9.jpg,82856-100088-906221-18123-10431-1077280-587092...,"['tomhanks', 'marianatreviño', 'rachelkeller',...",basedonnovelorbook suicideattempt remake misca...


In [4]:
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [5]:
def get_recommendations(title, mat):
    idx = indices[title]
    sim_scores = list(enumerate(mat[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    return [[titles.iloc[movie_indices[i]],movie_scores[i]] for i in range(len(movie_scores))]

In [6]:
def tfidf_cosine(df,col):
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(df[col])
    tfidf_matrix = tfidf_matrix.astype("float32")
    return linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
cosine_sim = tfidf_cosine(movies,"overview")

In [8]:
get_recommendations("The Dark Knight", cosine_sim)[:10]

[['Batman: The Long Halloween, Part One', 0.204439],
 ['Batman: The Long Halloween, Part Two', 0.1727246],
 ['The Dark Knight Rises', 0.16054228],
 ['Batman vs. Two-Face', 0.13137831],
 ['Batman: Gotham by Gaslight', 0.08755989],
 ['Batman', 0.08454768],
 ['Batman Forever', 0.08326309],
 ['Batman: Year One', 0.075660706],
 ['Batman: The Killing Joke', 0.06896779],
 ['JFK', 0.066365875]]

In [9]:
pop = pd.read_csv("popularity.csv")

In [10]:
pop["0"].dtype

dtype('float64')

### recommender function that filters out movies that arent really popular , where popularity is based off of a metric by tmdb

In [11]:
def also_get_recommendations(title, mat):
    idx = indices[title]
    sim_scores = list(enumerate(mat[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    recs = movies.iloc[movie_indices]
    threshold = pop["0"].quantile(0.60)
    
    index, qualified_titles, qualified_popularity = recs[(recs["popularity"] >= threshold)].index, recs[(recs["popularity"] >= threshold)]["title"],\
    recs[(recs["popularity"] >= threshold)]["popularity"]
    
    sim_scores = [sim_scores[i][1] for i in range(len(movie_indices)) if movie_indices[i] in index]
    
    return [[qualified_titles[index[i]],qualified_popularity[index[i]],sim_scores[i]] for i in range(len(index))]

In [16]:
def count_cosine(df,col):
    count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    count_matrix = count.fit_transform(df[col])
    count_matrix = count_matrix.astype("float32")
    count_cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    return count_cosine_sim

In [17]:
countCosineSim = count_cosine(movies,"soup")

# IMO, the "soup" gave better recommendations, and hence will be using it for the recommendation system

In [23]:
also_get_recommendations("Batman v Superman: Dawn of Justice",cosine_sim)

[['Justice League Dark', 19.204, 0.06272659],
 ['The Batman', 344.174, 0.0626558],
 ['Batman: The Long Halloween, Part Two', 23.778, 0.061375033],
 ['Batman: The Long Halloween, Part One', 30.35, 0.058979005],
 ['Kick-Ass', 40.013, 0.05808579],
 ['Batman & Robin', 31.212, 0.05754663],
 ['Batman: The Dark Knight Returns, Part 1', 32.236, 0.055963345],
 ['Teen Titans Go! & DC Super Hero Girls: Mayhem in the Multiverse',
  60.818,
  0.054138884],
 ['Tall Girl 2', 24.198, 0.053409003],
 ['The Dark Knight Rises', 77.329, 0.052511446],
 ['Batman: Bad Blood', 22.861, 0.051870927],
 ['Guardians', 45.784, 0.05149076],
 ['Batman: Under the Red Hood', 28.391, 0.048209745],
 ['Brightburn', 32.469, 0.046482407],
 ['Batman Begins', 53.942, 0.04459291],
 ['Spider-Man: No Way Home', 415.473, 0.043833543]]

In [25]:
also_get_recommendations("2001: A Space Odyssey", countCosineSim)

[['2010', 20.684, 0.29926604],
 ['Star Trek: The Motion Picture', 24.222, 0.20718417],
 ['Beneath the Planet of the Apes', 20.202, 0.19603921],
 ['Life', 26.886, 0.16568336],
 ['Star Wars: The Rise of Skywalker', 66.772, 0.16062313],
 ['Kill Command', 26.167, 0.156957],
 ['Allegiant', 28.211, 0.1560153],
 ['Gravity', 27.466, 0.1560153],
 ['The Matrix Resurrections', 122.226, 0.1494785],
 ["The Hitchhiker's Guide to the Galaxy", 23.006, 0.14611901],
 ['Transcendence', 19.617, 0.14611901],
 ['Barbarella', 24.049, 0.1459893],
 ['Planet of the Apes', 29.877, 0.1429763],
 ['Sunshine', 21.125, 0.14201431]]